Table 4.7.1: Sexual orientation, new presentations to treatment

In [ ]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [ ]:
tab = tabs['Table 4.7.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [ ]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()
observations

In [ ]:
Sexualorientation = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A13')
Sexualorientation

In [ ]:
Clients = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
Clients

In [ ]:
MeasureType = tab.excel_ref('B4').expand(RIGHT).is_not_blank()
MeasureType

In [ ]:
Dimensions = [
            HDim(Sexualorientation,'Sexual orientation',DIRECTLY,LEFT),
            HDim(Clients,'Clients',CLOSEST,LEFT),
            HDim(MeasureType,'Measure Type',DIRECTLY,ABOVE),
            HDimConst('Unit','People')            
            ]

In [ ]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [ ]:
new_table = c1.topandas()
new_table

In [ ]:
new_table = new_table[new_table['OBS'] != 0 ]

In [ ]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [ ]:
new_table['Measure Type'] = new_table['Measure Type'].map(
    lambda x: {
        'n' : 'Count', 
        '%' : 'Percentage',
        }.get(x, x))

In [ ]:
new_table.head()

In [ ]:
new_table.dtypes

In [ ]:
new_table['Value'] = new_table['Value'].astype(str)

In [ ]:
new_table.head(3)

In [ ]:
new_table['Sexual orientation'] = new_table['Sexual orientation'].map(
    lambda x: {
        'Total' : 'All' 
        }.get(x, x))

In [ ]:
new_table['Sexual orientation'].unique()

In [ ]:
def user_perc(x):
    
    if x == None :
        return 'All inclusice Inconsistent/missing'
    else:
        return x
    
new_table['Sexual orientation'] = new_table.apply(lambda row: user_perc(row['Sexual orientation']), axis = 1)

In [ ]:
new_table['Clients'] = new_table['Clients'].map(
    lambda x: {
        'Total' : 'All Clients',
        }.get(x, x))

In [ ]:
new_table.head(3)

In [ ]:
new_table['Sexual orientation'] = 'Sexual orientation/' + new_table['Sexual orientation']

In [ ]:
new_table['Clients'] = 'New clients/' + new_table['Clients']

In [ ]:
new_table.columns = ['Clients in treatment' if x=='Clients' else x for x in new_table.columns]
new_table['Period'] = '2017-18'
new_table.columns = ['Basis of treatment' if x=='Sexual orientation' else x for x in new_table.columns]
new_table['Substance'] = 'All'
new_table = new_table[['Period','Basis of treatment','Substance','Clients in treatment','Measure Type','Value','Unit']]

In [ ]:
new_table.head()